<a href="https://colab.research.google.com/github/azizhina51-svg/Linkedin-Goodies/blob/main/Linkedin_post_readibility_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Cell 1: install required libraries
!pip install -q textstat spacy sentence-transformers scikit-learn pandas


In [12]:
# Cell 2: download spaCy English model (run once)
!python -m spacy download en_core_web_sm -q


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
# Cell 3: imports and model initialization
import textstat
import spacy
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

# load models
nlp = spacy.load("en_core_web_sm")
sbert = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight SBERT


In [14]:
# Cell 4: basic_stats (tokenization, sentence counts, simple heuristics)
def basic_stats(text):
    doc = nlp(text)
    sentences = list(doc.sents)
    words = [token.text for token in doc if token.is_alpha]
    long_words = [w for w in words if len(w) > 6]
    avg_sentence_len = (len(words) / max(1, len(sentences)))
    short_sentence_ratio = sum(1 for s in sentences if len([t for t in s if t.is_alpha]) < 8) / max(1, len(sentences))
    return {
        'num_chars': len(text),
        'num_words': len(words),
        'num_sentences': len(sentences),
        'avg_sentence_len': avg_sentence_len,
        'pct_long_words': len(long_words) / max(1, len(words)),
        'short_sentence_ratio': short_sentence_ratio
    }


In [15]:
# Cell 5: compute classic readability formulas via textstat
def readability_metrics(text):
    return {
        'flesch_reading_ease': textstat.flesch_reading_ease(text),
        'flesch_kincaid_grade': textstat.flesch_kincaid_grade(text),
        'gunning_fog': textstat.gunning_fog(text),
        'smog_index': textstat.smog_index(text),
        'automated_readability_index': textstat.automated_readability_index(text),
        'dale_chall_readability_score': textstat.dale_chall_readability_score(text)
    }

def normalize_scores(scores_dict):
    # Convert different metrics into 0-100 ease score (higher = easier)
    out = {}
    out['flesch_reading_ease_norm'] = np.clip(scores_dict['flesch_reading_ease'], 0, 100)
    def invert_grade(v, max_grade=20):
        v = max(0, min(v, max_grade))
        return (1 - (v / max_grade)) * 100
    out['flesch_kincaid_grade_norm'] = invert_grade(scores_dict['flesch_kincaid_grade'])
    out['gunning_fog_norm'] = invert_grade(scores_dict['gunning_fog'])
    out['smog_index_norm'] = invert_grade(scores_dict['smog_index'])
    out['ari_norm'] = invert_grade(scores_dict['automated_readability_index'])
    dc = scores_dict['dale_chall_readability_score']
    dc = max(4.0, min(dc, 10.0))
    out['dale_chall_norm'] = (1 - ((dc - 4.0) / (10.0 - 4.0))) * 100
    return out


In [16]:
# Cell 6: composite score combining normalized formulas + heuristics
def composite_readability(text, weights=None):
    if weights is None:
        weights = {
            'flesch_reading_ease_norm': 1,
            'flesch_kincaid_grade_norm': 1,
            'gunning_fog_norm': 1,
            'smog_index_norm': 1,
            'ari_norm': 1,
            'dale_chall_norm': 1
        }
    r = readability_metrics(text)
    rn = normalize_scores(r)
    stats = basic_stats(text)
    short_sentence_bonus = max(0, (1 - stats['short_sentence_ratio'])) * 10
    long_word_penalty = (1 - (1 - stats['pct_long_words'])) * -5
    keys = list(weights.keys())
    vals = np.array([rn[k] for k in keys], dtype=float)
    w = np.array([weights[k] for k in keys], dtype=float)
    w = w / w.sum()
    core = (vals * w).sum()
    composite = core + short_sentence_bonus + long_word_penalty
    composite = float(np.clip(composite, 0, 100))
    return {
        'composite_score': composite,
        'component_norms': rn,
        'heuristics': stats,
        'raw_metrics': r
    }


In [20]:
# Cell 7: quick demo using a sample text (run to see outputs)
sample_text = """Writing for LinkedIn is a mix of clarity and human voice.
Use short sentences, avoid jargon, and give a single clear idea per post.
Bulleted lists and emojis can help scannability, but too many links hurt readability."""

result = composite_readability(sample_text)
print("COMPOSITE READABILITY (0=hard -> 100=super easy):", result['composite_score'])
print("\nNormalized component scores:")
for k,v in result['component_norms'].items():
    print(f"  {k}: {v:.1f}")
print("\nHeuristic stats:")
for k,v in result['heuristics'].items():
    print(f"  {k}: {v:.3f}")
print("\nRaw formula metrics (for debug):")
for k,v in result['raw_metrics'].items():
    print(f"  {k}: {v}")


COMPOSITE READABILITY (0=hard -> 100=super easy): 55.70255078667961

Normalized component scores:
  flesch_reading_ease_norm: 61.7
  flesch_kincaid_grade_norm: 61.4
  gunning_fog_norm: 48.3
  smog_index_norm: 44.0
  ari_norm: 61.1
  dale_chall_norm: 3.4

Heuristic stats:
  num_chars: 217.000
  num_words: 37.000
  num_sentences: 2.000
  avg_sentence_len: 18.500
  pct_long_words: 0.189
  short_sentence_ratio: 0.000

Raw formula metrics (for debug):
  flesch_reading_ease: 61.70045045045046
  flesch_kincaid_grade: 7.717297297297296
  gunning_fog: 10.33873873873874
  smog_index: 11.20814326018867
  automated_readability_index: 7.777477477477479
  dale_chall_readability_score: 9.796071171171171


In [22]:
sample_text = """Why do I need you? All the info I need is available online.” 🤷🏼‍♀️

This is a common question I get from professionals who want to build & monetise a personal brand,
but are unsure about my Executive Branding Program.
-
My response to this?

This is true.
Everything is online.

But information ISN’T what gets you results.
It’s so much more than that.

My Executive Branding Program works because it gives you everything the internet can’t.

It also:
1️⃣ Holds you accountable,
so you do the work between sessions,
and don’t let it slip when your 9-5 gets too busy


2️⃣ Brings real-life experience & answers from someone who has
LIVED through it & created systems that WORK for your exact situation

(e.g., how to fit everything in when you work a 9-5, 60+ hours a week)


3️⃣ Works on not just technical knowledge,
but the IDENTITY shifts that are required to take
your online business and presence where you want to go

(e.g., moving past the fear of being seen, dealing with the potential judgment from family & friends)


4️⃣ Being able to provide an objective perspective,
when reviewing your positioning, content, and overall strategy,
because it’s now second nature to me & so much easier to pick up on things the untrained eye doesn’t see

(e.g., is your post audience-focused? Or just about you?
But then, is there enough of your personality injected into it too?)


5️⃣ Being able to access ALL of the information at the right time,
and focusing/deep-diving on where you need the most support,
or where you could move much faster


6️⃣ Having someone that believes in you, supports you,
and gives you the confidence to show up online,
long before you believe it yourself

(Something I wish I had long long ago)


So yeah, you CAN find all the technical knowledge online.
But you won’t get the speed, clarity, confidence, and personalised strategy,
that only comes with experience & talking to a real person 🤝🏼."""

result = composite_readability(sample_text)
print("COMPOSITE READABILITY (0=hard -> 100=super easy):", result['composite_score'])
print("\nNormalized component scores:")
for k,v in result['component_norms'].items():
    print(f"  {k}: {v:.1f}")
print("\nHeuristic stats:")
for k,v in result['heuristics'].items():
    print(f"  {k}: {v:.3f}")
print("\nRaw formula metrics (for debug):")
for k,v in result['raw_metrics'].items():
    print(f"  {k}: {v}")

COMPOSITE READABILITY (0=hard -> 100=super easy): 45.853699551784736

Normalized component scores:
  flesch_reading_ease_norm: 60.6
  flesch_kincaid_grade_norm: 52.3
  gunning_fog_norm: 39.7
  smog_index_norm: 40.5
  ari_norm: 48.5
  dale_chall_norm: 8.3

Heuristic stats:
  num_chars: 1932.000
  num_words: 315.000
  num_sentences: 15.000
  avg_sentence_len: 21.000
  pct_long_words: 0.225
  short_sentence_ratio: 0.467

Raw formula metrics (for debug):
  flesch_reading_ease: 60.590294117647076
  flesch_kincaid_grade: 9.542941176470592
  gunning_fog: 12.06797385620915
  smog_index: 11.892052765847286
  automated_readability_index: 10.291573926868047
  dale_chall_readability_score: 9.504009005083514


In [25]:
sample_text = """Most of you treat LinkedIn like it’s a Hogwarts potions exam and Professor Snape is glaring at you from two feet away.

Drop 17 comments before posting.
Reply to all recent DMs.
Add a bit of bezoar.
Mix it with some murtlap tentacles.
Finish it off with mistletoe berries.

Seriously, please stop.

Here are 4 things that actually move the needle:

1. Write interesting posts that actually feel like they were written by a living, breathing human. (Or get someone to do it for you)

2. Add relevant people to your network. (Creators and target audience)

3. Spend time in the DMs. Get those conversations going.

4. Support your peers via comments.

That's it. Everything else is just noise that is distracting you from the real stuff.

Take my advice, keep things simple. Simple works.

(If you don't believe me, just ask Harry)
"""

result = composite_readability(sample_text)

print("COMPOSITE READABILITY (0=hard -> 100=super easy):", result['composite_score'])
print("\nNormalized component scores:")
for k, v in result['component_norms'].items():
    print(f"  {k}: {v:.1f}")

print("\nHeuristic stats:")
for k, v in result['heuristics'].items():
    print(f"  {k}: {v:.3f}")

print("\nRaw formula metrics (for debug):")
for k, v in result['raw_metrics'].items():
    print(f"  {k}: {v}")

COMPOSITE READABILITY (0=hard -> 100=super easy): 65.25297645711399

Normalized component scores:
  flesch_reading_ease_norm: 77.5
  flesch_kincaid_grade_norm: 77.9
  gunning_fog_norm: 68.7
  smog_index_norm: 60.1
  ari_norm: 75.8
  dale_chall_norm: 16.7

Heuristic stats:
  num_chars: 833.000
  num_words: 137.000
  num_sentences: 20.000
  avg_sentence_len: 6.850
  pct_long_words: 0.204
  short_sentence_ratio: 0.650

Raw formula metrics (for debug):
  flesch_reading_ease: 77.49166860916864
  flesch_kincaid_grade: 4.42441724941725
  gunning_fog: 6.254700854700855
  smog_index: 7.984000788550335
  automated_readability_index: 4.8406138306138295
  dale_chall_readability_score: 8.999425563325564


In [21]:
# Cell 8: optional - compute SBERT embedding for downstream ML
emb = sbert.encode([sample_text])[0]
print("Sentence embedding vector length:", len(emb))
# Save embedding to a DataFrame for example
df_emb = pd.DataFrame([emb])
df_emb.to_csv("sample_embedding.csv", index=False)
print("Saved embedding to sample_embedding.csv")


Sentence embedding vector length: 384
Saved embedding to sample_embedding.csv


In [29]:
import streamlit as st
import textstat
import spacy
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load NLP and embedding models
nlp = spacy.load("en_core_web_sm")
sbert = SentenceTransformer('all-MiniLM-L6-v2')


In [30]:
def basic_stats(text):
    doc = nlp(text)
    sentences = list(doc.sents)
    words = [token.text for token in doc if token.is_alpha]
    long_words = [w for w in words if len(w) > 6]
    avg_sentence_len = (len(words) / max(1, len(sentences)))
    short_sentence_ratio = sum(1 for s in sentences if len([t for t in s if t.is_alpha]) < 8) / max(1, len(sentences))
    return {
        'num_chars': len(text),
        'num_words': len(words),
        'num_sentences': len(sentences),
        'avg_sentence_len': avg_sentence_len,
        'pct_long_words': len(long_words) / max(1, len(words)),
        'short_sentence_ratio': short_sentence_ratio
    }

def readability_metrics(text):
    return {
        'flesch_reading_ease': textstat.flesch_reading_ease(text),
        'flesch_kincaid_grade': textstat.flesch_kincaid_grade(text),
        'gunning_fog': textstat.gunning_fog(text),
        'smog_index': textstat.smog_index(text),
        'automated_readability_index': textstat.automated_readability_index(text),
        'dale_chall_readability_score': textstat.dale_chall_readability_score(text)
    }

def normalize_scores(scores_dict):
    out = {}
    out['flesch_reading_ease_norm'] = np.clip(scores_dict['flesch_reading_ease'], 0, 100)
    def invert_grade(v, max_grade=20):
        v = max(0, min(v, max_grade))
        return (1 - (v / max_grade)) * 100
    out['flesch_kincaid_grade_norm'] = invert_grade(scores_dict['flesch_kincaid_grade'])
    out['gunning_fog_norm'] = invert_grade(scores_dict['gunning_fog'])
    out['smog_index_norm'] = invert_grade(scores_dict['smog_index'])
    out['ari_norm'] = invert_grade(scores_dict['automated_readability_index'])
    dc = scores_dict['dale_chall_readability_score']
    dc = max(4.0, min(dc, 10.0))
    out['dale_chall_norm'] = (1 - ((dc - 4.0) / (10.0 - 4.0))) * 100
    return out

def composite_readability(text, weights=None):
    if weights is None:
        weights = {
            'flesch_reading_ease_norm': 1,
            'flesch_kincaid_grade_norm': 1,
            'gunning_fog_norm': 1,
            'smog_index_norm': 1,
            'ari_norm': 1,
            'dale_chall_norm': 1
        }
    r = readability_metrics(text)
    rn = normalize_scores(r)
    stats = basic_stats(text)
    short_sentence_bonus = max(0, (1 - stats['short_sentence_ratio'])) * 10
    long_word_penalty = (1 - (1 - stats['pct_long_words'])) * -5
    keys = list(weights.keys())
    vals = np.array([rn[k] for k in keys], dtype=float)
    w = np.array([weights[k] for k in keys], dtype=float)
    w = w / w.sum()
    core = (vals * w).sum()
    composite = core + short_sentence_bonus + long_word_penalty
    composite = float(np.clip(composite, 0, 100))
    return {
        'composite_score': composite,
        'component_norms': rn,
        'heuristics': stats,
        'raw_metrics': r
    }


In [31]:
st.title("LinkedIn Post Readability Tool")
st.write("Paste your LinkedIn post below to see the readability analysis.")

user_input = st.text_area("Your LinkedIn Post", height=250)


2025-12-09 07:38:07.712 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:07.714 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:07.715 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:07.716 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:07.717 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:07.719 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:07.721 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:07.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [32]:
if st.button("Analyze"):
    if len(user_input.strip()) == 0:
        st.warning("Please enter some text first!")
    else:
        result = composite_readability(user_input)
        st.subheader(f"Composite Readability Score: {result['composite_score']:.1f}/100")

        # Show normalized component scores as a bar chart
        comp_df = pd.DataFrame(result['component_norms'], index=["Score"]).T
        st.bar_chart(comp_df)

        # Show heuristic stats as a table
        heur_df = pd.DataFrame(result['heuristics'], index=["Value"]).T
        st.subheader("Heuristic Stats")
        st.table(heur_df)

        # Optional: embedding vector
        emb = sbert.encode([user_input])[0]
        st.write(f"Sentence embedding vector length: {len(emb)}")


2025-12-09 07:38:31.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:31.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:31.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:31.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:31.373 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-09 07:38:31.374 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [33]:
!pip install pyngrok


In [39]:
%%writefile app.py
import streamlit as st
import textstat
import spacy
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load models
nlp = spacy.load("en_core_web_sm")
sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Helper functions
def basic_stats(text):
    doc = nlp(text)
    sentences = list(doc.sents)
    words = [token.text for token in doc if token.is_alpha]
    long_words = [w for w in words if len(w) > 6]
    avg_sentence_len = (len(words) / max(1, len(sentences)))
    short_sentence_ratio = sum(1 for s in sentences if len([t for t in s if t.is_alpha]) < 8) / max(1, len(sentences))
    return {
        'num_chars': len(text),
        'num_words': len(words),
        'num_sentences': len(sentences),
        'avg_sentence_len': avg_sentence_len,
        'pct_long_words': len(long_words) / max(1, len(words)),
        'short_sentence_ratio': short_sentence_ratio
    }

def readability_metrics(text):
    return {
        'flesch_reading_ease': textstat.flesch_reading_ease(text),
        'flesch_kincaid_grade': textstat.flesch_kincaid_grade(text),
        'gunning_fog': textstat.gunning_fog(text),
        'smog_index': textstat.smog_index(text),
        'automated_readability_index': textstat.automated_readability_index(text),
        'dale_chall_readability_score': textstat.dale_chall_readability_score(text)
    }

def normalize_scores(scores_dict):
    out = {}
    out['flesch_reading_ease_norm'] = np.clip(scores_dict['flesch_reading_ease'], 0, 100)
    def invert_grade(v, max_grade=20):
        v = max(0, min(v, max_grade))
        return (1 - (v / max_grade)) * 100
    out['flesch_kincaid_grade_norm'] = invert_grade(scores_dict['flesch_kincaid_grade'])
    out['gunning_fog_norm'] = invert_grade(scores_dict['gunning_fog'])
    out['smog_index_norm'] = invert_grade(scores_dict['smog_index'])
    out['ari_norm'] = invert_grade(scores_dict['automated_readability_index'])
    dc = scores_dict['dale_chall_readability_score']
    dc = max(4.0, min(dc, 10.0))
    out['dale_chall_norm'] = (1 - ((dc - 4.0) / (10.0 - 4.0))) * 100
    return out

def composite_readability(text, weights=None):
    if weights is None:
        weights = {
            'flesch_reading_ease_norm': 1,
            'flesch_kincaid_grade_norm': 1,
            'gunning_fog_norm': 1,
            'smog_index_norm': 1,
            'ari_norm': 1,
            'dale_chall_norm': 1
        }
    r = readability_metrics(text)
    rn = normalize_scores(r)
    stats = basic_stats(text)
    short_sentence_bonus = max(0, (1 - stats['short_sentence_ratio'])) * 10
    long_word_penalty = (1 - (1 - stats['pct_long_words'])) * -5
    keys = list(weights.keys())
    vals = np.array([rn[k] for k in keys], dtype=float)
    w = np.array([weights[k] for k in keys], dtype=float)
    w = w / w.sum()
    core = (vals * w).sum()
    composite = core + short_sentence_bonus + long_word_penalty
    composite = float(np.clip(composite, 0, 100))
    return {
        'composite_score': composite,
        'component_norms': rn,
        'heuristics': stats,
        'raw_metrics': r
    }

# Streamlit UI
st.title("LinkedIn Post Readability Tool")
st.write("Paste your LinkedIn post below to see the readability analysis.")

user_input = st.text_area("Your LinkedIn Post", height=250)

if st.button("Analyze"):
    if len(user_input.strip()) == 0:
        st.warning("Please enter some text first!")
    else:
        result = composite_readability(user_input)
        st.subheader(f"Composite Readability Score: {result['composite_score']:.1f}/100")

        # Show normalized component scores as a bar chart
        comp_df = pd.DataFrame(result['component_norms'], index=["Score"]).T
        st.bar_chart(comp_df)

        # Show heuristic stats as a table
        heur_df = pd.DataFrame(result['heuristics'], index=["Value"]).T
        st.subheader("Heuristic Stats")
        st.table(heur_df)

        # Optional: embedding vector
        emb = sbert.encode([user_input])[0]
        st.write(f"Sentence embedding vector length: {len(emb)}")


Overwriting app.py


In [51]:
# Install pyngrok if not already installed
!pip install -q pyngrok

# Import ngrok
from pyngrok import ngrok

# Set your ngrok auth token here
NGROK_AUTH_TOKEN = "36VZIQ8m0uq9JSWzt6NpbL9wD1k_7kASWUcjeHJxbnjqExtad"  # <-- Replace with your token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Optional: test connection by opening a tunnel (not required if you already ran it)
public_url = ngrok.connect(8501)
print("Streamlit public URL:", public_url)


Streamlit public URL: NgrokTunnel: "https://unindustrialized-iesha-uncomplimented.ngrok-free.dev" -> "http://localhost:8501"
